In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:77% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import pickle
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from collections import OrderedDict
from net.st_gcn import Model
from feeder.feeder import Feeder
from torchlight.io import IO
np.set_printoptions(linewidth=100)

In [3]:
graph_args = {'layout':'openpose', 'strategy':'spatial'}

In [4]:
# Loading of original weights
root_path = '/root/sharedfolder/Research/pose_ad/st-gcn/'
io = IO(root_path)
model_name = 'net.st_gcn.Model'
model_args = {
    'edge_importance_weighting':True,
    'graph_args':graph_args,
    'in_channels':3,
    'num_class':400, }
model_args_ft = {
    'edge_importance_weighting':True,
    'graph_args':graph_args,
    'in_channels':3,
    'num_class':4, }

In [5]:
model_fio = io.load_model(model_name, **model_args)
model_2ft = io.load_model(model_name, **model_args_ft)

In [6]:
weights_path =  os.path.join(root_path, 'models/kinetics-st_gcn.pt')

In [ ]:
weights = torch.load(weights_path)
weights_list = [[k.split('module.')[-1], v.cpu()] for k, v in weights.items()]
weights = OrderedDict(weights_list)
model_state_dict = model_2ft.state_dict()
weights.pop('fcn.bias') # loading all but the Final FC layer's weight and bias
weights.pop('fcn.weight')
model_state_dict.update(weights)
model_2ft.load_state_dict(model_state_dict)

In [ ]:
# last_fcn_weights_str = ['fcn.weigth', 'fcn.bias']
# model_fio = io.load_weights(model_fio, weights_path)
# model_2ft = io.load_weights(model_2ft, weights_path)

In [ ]:
model_2ft.parameters

In [ ]:
# Altered Model Definition
new_class_num = 4

# model_minus_last_list = list(model_2ft.children())[:-1]
# model_minus_last_list.extend([nn.Conv2d(256, new_class_num, kernel_size=1)])
# model_2ft = nn.Sequential(*model_minus_last_list)

In [ ]:
# Comparing weights of the two models to verify removal of the last layer didn't harm the weights

# orig_last_stgcn = model_fio.st_gcn_networks[9].parameters()
# orig_weights_mat = torch.ones((1, 256))
# for param in orig_last_stgcn:
# #     print(param.data.shape)
#     data256 = param.data.view((-1, 256))
# #     print(data256.shape)
#     orig_weights_mat = torch.cat((orig_weights_mat, data256), 0)
# orig_weights_mat.shape

# ft_last_stgcn = model_fio.st_gcn_networks[9].parameters()
# ft_weights_mat = torch.ones((1, 256))
# for param in ft_last_stgcn:
# #     print(param.data.shape)
#     data256 = param.data.view((-1, 256))
# #     print(data256.shape)
#     ft_weights_mat = torch.cat((ft_weights_mat, data256), 0)
# ft_weights_mat.shape

# ## Equivalence test
# np.allclose(ft_weights_mat, orig_weights_mat)

In [ ]:
# Create a function for the cel below
def loader_initializer(feeder_args_dict, batch_size=32, suffle=True, num_workers=4, drop_last=True):
    data_loader = torch.utils.data.DataLoader(
    dataset=Feeder(**feeder_args_dict),
    batch_size=batch_size,
    shuffle=suffle,
    num_workers=num_workers,
    drop_last=drop_last)
    return data_loader

In [ ]:
data_dir_path = '/root/sharedfolder/datasets/data_ssd/kinetics-skeleton/st-gcn_kinetics/Kinetics/kinetics-skeleton/'
train_data_path = os.path.join(data_dir_path, 'train_data.npy')
train_label_path = os.path.join(data_dir_path, 'train_label.pkl')
test_data_path = os.path.join(data_dir_path, 'val_data.npy')
test_label_path = os.path.join(data_dir_path, 'val_label.pkl')

test_feeder_args = {'data_path': test_data_path, 'label_path': test_label_path}
train_feeder_args = {'data_path': train_data_path, 'label_path': train_label_path}

batch_size=32
data_loader = dict()
data_loader['kinetics_train'] = loader_initializer(train_feeder_args, batch_size=batch_size)
data_loader['kinetics_test' ] = loader_initializer(test_feeder_args, batch_size=batch_size)

In [ ]:
train_npy = np.load(train_data_path, mmap_mode='r')
test_npy = np.load(test_data_path, mmap_mode='r')

In [ ]:
save_files = False
gen_train = False
if gen_train:
    split_npy = train_npy
    split_2gen = 'normal_train'
    f = train_label_path
else:
    split_npy = test_npy
    split_2gen = 'normal_test'
    f = test_label_path

In [ ]:
data_path = dict()
label_path = dict()
split_indices = dict()
data_npy = dict()
data_pkl = dict()  # An array of [sample_name, label]


In [ ]:
# Train Split Generator
data_pkl['normal_train'] = [[],[]]  # Important to zero so not to accumulate older values
data_pkl['normal_test'] = [[],[]]
for split in ['front_raises134', 'deadlifting88', 'clean_jerk59', 'forcast254', 'normal_train', 'normal_test', 'abnormal_test']:
    data_path[split]  =  os.path.join(data_dir_path, 'ad_experiment', split+'_data.npy')
    label_path[split] =  os.path.join(data_dir_path, 'ad_experiment', split+'_label.pkl')
    split_num = re.findall('\d+', split)
    if split_num != [] and save_files:
        split_indices[split] = [ind for ind, val in enumerate(label) if val == int(split_num[0])]
        data_npy[split] = split_npy[split_indices[split]]
        data_pkl[split] = [[sample_name[i] for i in split_indices[split]], [label[i] for i in split_indices[split]]]
        data_pkl[split_2gen][0] += data_pkl[split][0]
        data_pkl[split_2gen][1] += data_pkl[split][1]
        print("Split {}, data_pkl[split][0] len={}, data_pkl[split_2gen][0] len={}".format(split, len(data_pkl[split][0]), len(data_pkl[split_2gen][0])))

# data_npy[split_2gen] = np.concatenate((data_npy['front_raises134'], data_npy['deadlifting88'], data_npy['clean_jerk59'], data_npy['forcast254']))

In [ ]:
[134, 88, 59, 254].index(134)

In [ ]:
if save_files and gen_train:
    np.save(data_path['normal_train'], data_npy['normal_train'])
    pickle.dump(data_pkl['normal_train'], open(label_path['normal_train'] ,"wb"))
elif save_files:
    np.save(data_path['normal_test'], data_npy['normal_test'])
    pickle.dump(data_pkl['normal_test'], open(label_path['normal_test'] ,"wb"))

In [ ]:
# test_feeder_args = {'data_path': test_data_path, 'label_path': test_label_path}
normal_train_feeder_args = {'data_path': data_path['normal_train'], 'label_path': label_path['normal_train']}
normal_test_feeder_args = {'data_path': data_path['normal_test'], 'label_path': label_path['normal_test']}

batch_size=32
data_loader['normal_train'] = loader_initializer(normal_train_feeder_args, batch_size=batch_size, num_workers=0) # , drop_last=False)
data_loader['normal_test' ] = loader_initializer(normal_test_feeder_args, batch_size=batch_size, num_workers=0) #, drop_last=False)

In [ ]:
dev = 'cuda:0'
model = model_2ft.cuda()
loss_fn = nn.CrossEntropyLoss()
loader = data_loader['normal_train']

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
# Freezing all but the new layer
for name, param in model.named_parameters():
    param.requires_grad = False
    if "fcn" in name:
        param.requires_grad = True

In [ ]:
# # loading files manually for verifying the loader
# fdata = np.load(normal_train_feeder_args['data_path'], mmap_mode='r')
# fsample_name, flabel = pickle.load(open(normal_train_feeder_args['label_path'], "rb"))

# fdata = np.load(normal_test_feeder_args['data_path'], mmap_mode='r')
# fsample_name, flabel = pickle.load(open(normal_test_feeder_args['label_path'], "rb"))

# print(fdata.shape, len(fsample_name), len(flabel))

In [ ]:
# Training Loop
for data, label in loader:
            # get data
            data = data.float().to(dev)
            label = label.long().to(dev)

            # forward
            output = model(data)
            loss = loss_fn(output, label)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print("Completed iteration")
            # statistics
#             self.iter_info['loss'] = loss.data.item()
#             self.iter_info['lr'] = '{:.6f}'.format(self.lr)
#             loss_value.append(self.iter_info['loss'])
#             self.show_iter_info()
#             self.meta_info['iter'] += 1